### json 불러와서 저장하기

In [69]:
import numpy as np
import pandas as pd
import json
import jpype
import glob
from random import shuffle
from bs4 import BeautifulSoup as bs

data_path = "data"
# data_path = "data(test)"

file_list = glob.glob("%s/*.json" % data_path)

shuffle(file_list)

json_train=[]

for json_file_name in file_list:
    json_file = json.loads(open(json_file_name).read())
    json_train += json_file["articles"]

### json에서 title 형태소 추출

In [70]:
from konlpy.tag import Mecab

mecab = Mecab()

labeled_train = []

for cnt, article in enumerate(json_train):
    
    if cnt % 10000 == 0:
        print cnt

    labeled_train.append({
            "istroll": article["is_troll"],
            "title_pos": ["%s_%s" % (first, second) for first, second in mecab.pos(article["title"])]
        })

labeled_train = pd.DataFrame.from_dict(labeled_train)
labeled_train

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000


,istroll,title_pos
0,False,"[김연아_NNP, 는_JX, 이미_MAG, 올림픽_NNP, 을_JKO, 뛰어넘_VV..."
1,True,"[한국_NNP, 과_JC, 일본_NNP, 에_JKB, 만_JX, 있_VX, 는_ET..."
2,False,"[(_SSO, 영상_NNG, )_SSC, 박근혜_NNP, 정부_NNG, 출범_NNG..."
3,False,"[석_MM, 희_NNG, 의_JKG, 그림_NNG, 공부_NNG, ._SF]"
4,False,"[최고_NNG, 의_JKG, 라면_NNG, 은_JX]"
5,False,"[글_NNG, 써도_VV+EC, 되_VV+EC, 농_NNG, ?_SY, 2004_S..."
6,True,"[와우_NNG, ~_SY, 굿_NNG, 정보_NNG, !_SF, !!!_SY]"
7,True,"[함_MAG, 둘러보_VV, 세요_EP+EF, ~_SY]"
8,False,"[[_SSO, 롤_NNG, ]_SSC, 일_VCP+ETM, 워_NNG, 채팅_NNG..."
9,True,"[이것_NP, 도_JX, 인연_NNG, 인데_VCP+EF, ._SF, ._SF]"


###bag of words

In [71]:
from sklearn.feature_extraction.text import CountVectorizer
def make_bag_of_words(labeled_train, max_features, col_name):
    vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None,
                                stop_words = None, max_features=max_features)

    train_data_features = vectorizer.fit_transform((labeled_train[col_name].apply(lambda x:" ".join(x)))).toarray()

    col = ["bow_%s_%s" % (col_name, data) for data in vectorizer.get_feature_names()]
    labeled_train = pd.concat([labeled_train, pd.DataFrame(train_data_features, columns = col, index=labeled_train.index)],axis=1)
    
    return labeled_train

##LDA

In [72]:
import gensim
from gensim import corpora, models
from gensim.corpora import TextCorpus, MmCorpus, Dictionary
from nltk.corpus import stopwords
from optparse import OptionParser

def make_lda(train, keep_n, num_topics, col_name):

    data = train[col_name]

    dictionary = corpora.Dictionary(data)
    dictionary.filter_extremes(keep_n=keep_n)

    corpus = [dictionary.doc2bow(text) for text in data]

    print("Make Lda..")

#    lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, chunksize=1000, passes=1,distributed = True)
    lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, chunksize=1000, passes=1)
    
    num = len(train)    
    df = []
    for i in range(0,num):
        if i%10000 == 0: print(i)
        temp = [i[1] for i in lda.get_document_topics(corpus[i],minimum_probability=0)]
        #temp = max(lda.get_document_topics(corpus[i]),key=lambda item:item[1])[0]
        df.append(temp)
    
    col = ["lda_%s_%d" % (col_name, data) for data in range(0, num_topics)]
    df = pd.DataFrame(df, columns = col)
    df.index = index
    
    train = pd.concat([train,df],axis=1)
        
    return train

In [73]:
max_features = 300
#%time labeled_train = make_lda(labeled_train, 5000, 1000, "title_pos")
%time labeled_train = make_bag_of_words(labeled_train, 300, "title_pos")

CPU times: user 3.04 s, sys: 413 ms, total: 3.45 s
Wall time: 3.43 s


### predictor, model 생성

In [74]:
from sklearn.ensemble import RandomForestClassifier

label = 'istroll'
pre = labeled_train.columns.drop(['title_pos', label])

model = RandomForestClassifier(n_estimators=10, n_jobs=6)

###cross validation

In [75]:
from sklearn import cross_validation

cv_value = 3

scores = cross_validation.cross_val_score(model, labeled_train[pre], labeled_train[label], cv=cv_value, scoring="roc_auc")
cv_result = scores.mean()

print(cv_result)

0.647444316789


##Result
  * title의 morphs를 bag of words로 변환 (feature : 1,000개) - 0.612752193303
  * title의 pos를 bag of words로 변환 (feature : 1,000개) - 0.630053006037
  * 데이터를 셔플하고 title의 pos를 bag of words로 변환 (feature : 1,000개) - 0.658894903487
  
  * title의 pos를 LDA로 변환 (keep_n : 5,000, num_topics : 1,000) - 0.592446285012

In [76]:
import pickle

pickle.dump(labeled_train[pre], open("title_%d.p" % max_features, "wb"), protocol = pickle.HIGHEST_PROTOCOL)